In [ ]:
import random
#from operator import attrgetter

kolory=["♠️","♥️","♣️","♦️"]
wartości=["A","K","Q","W","10","9","8","7","6","5","4","3","2"]
pozycje=["N","E","S","W"]

#awersy
piki=['🂡', '🂢', '🂣', '🂤', '🂥', '🂦', '🂧', '🂨', '🂩', '🂪', '🂫', '🂭', '🂮']
kiery=['🂱', '🂲', '🂳', '🂴', '🂵', '🂶', '🂷', '🂸', '🂹', '🂺', '🂻', '🂽', '🂾']
karo=['🃁', '🃂', '🃃', '🃄', '🃅', '🃆', '🃇', '🃈', '🃉', '🃊', '🃋', '🃍', '🃎']
trefle=['🃑', '🃒', '🃓', '🃔', '🃕', '🃖', '🃗', '🃘', '🃙', '🃚', '🃛', '🃝', '🃞']
jokery=['🂿', '🃏']

class Karta():
    '''definicja karty do gry''' 
    def __init__(self, wartość: str = "", kolor: str = ""):
        self.kolor = kolor
        self.wartość = wartość

    def __str__(self)->str:
        return self.get_label()
    
    def get_label(self)->str:
        return self.wartość + self.kolor
    
class CardsCollection(list[Karta]):
    '''definicja stosu kart'''
    def __init__(self, *args: Karta|None):
        '''utworzenie stosu kart'''
        super().__init__(*args)
    
    def __str__(self):
        return f'{self.get_labels(self)}'
    
    def w_stosie(self:CardsCollection)->list[str]:
        nazwy_kart:list[str]=[]
        for karta in self:
            nazwy_kart.append(karta.get_label())
        return(nazwy_kart)
    
    def liczba_kart(self)->int:
        '''liczba kart w stosie'''
        return len(self)
    
    @staticmethod
    def get_labels(stack: CardsCollection|list[Karta])->list[str]:
        '''lista nazw kart w stosie'''
        nazwy_kart:list[str]=[]
        for karta in stack:
            nazwy_kart.append(karta.get_label())
        return(nazwy_kart)
 
class Talia(CardsCollection):
    '''definicja talii kart do gry'''
    
    def __init__(self, *args: Karta|None):
        '''utworzenie talii kart'''
        super().__init__(*args)
        for kolor in kolory:
            for wartość in wartości:
                self.append(Karta(wartość,kolor))
    
    def __str__(self):
        return f'{self.get_labels(self)}'
    
    def randomize_cards(self):
        '''tasowanie talii'''
        random.shuffle(self)

    def tasuj_fisher_yates(self):
        '''Fisher-Yates shuffle - algorytm tasowania O(n)'''
        for i in range(len(self) - 1, 0, -1):   # Iteracja od ostatniego elementu do pierwszego
            j = random.randint(0, i)            # Losowy indeks od 0 do bieżącego indeksu (i)
            self[i], self[j] = self[j], self[i] # Zamiana elementów (swap)

    def get_random_card(self)->Karta|None:
        '''losowanie karty z talii'''

        random_card: Karta|None = None
        if len(self) > 0:
            random_card = random.choice(self)
            self.remove(random_card)
        return(random_card)
    
    def get_first_card(self)->Karta|None:
        '''pobranie pierwszej karty z talii'''

        first_card: Karta|None = None
        if len(self) > 0:
            first_card = self.pop(0)
        return(first_card)
    
class Osoba():
    '''definicja osoby'''

    def __init__(self, imię: str):
        self.imię = imię
    
    def __str__(self):
        return f'{self.imię}'
   
class Gracz(CardsCollection, Osoba):
    '''definicja grającego'''

    def __init__(self, *args: Karta|None, imię: str|None = None, pozycja: str|None = None):
        super().__init__(*args)  
        self.imię = imię
        self.pozycja = pozycja
    
    def __str__(self):
        return f'{self.imię}: {self.hand()}'
    
    def store_card(self, karta: Karta|None):
        if karta is not None:
            self.append(karta)
        
    def hand(self)->list[str]:
        return(self.get_labels(self))
    
    def hand_sorted(self)->list[str]:
        posortowane:list[Karta]=sorted(
            self,
            key=lambda karta:(kolory.index(karta.kolor), wartości.index(karta.wartość)),
            reverse=False
        )
        return(self.get_labels(posortowane))
    
class Players(list[Gracz]):
    '''definicja grupy grających'''

    def __init__(self, *args: Gracz|None, deck: Talia|None = None):
        super().__init__(*args)
        self.deck:Talia = deck if deck is not None else Talia()
        self.deck
    
    def rozdanie(self):
        for _ in range(13): #rozdanie 13 kart każdemu z grających
            for gracz in self:
                gracz.store_card(self.deck.get_first_card())

In [ ]:
# obsadzenie stołu i rozdanie kart
the_table = Players(deck=Talia())
the_table.append(Gracz(imię="Piotr",pozycja="N"))
the_table.append(Gracz(imię="Agata",pozycja="E"))
the_table.append(Gracz(imię="Kasia",pozycja="S"))
the_table.append(Gracz(imię="Karol",pozycja="W"))

# tasowanie talii
the_table.deck.randomize_cards()
the_table.deck.tasuj_fisher_yates()
#print(f'talia po tasowaniu:     {the_table.deck}, jest: {the_table.deck.liczba_kart()}')

# rozdanie kart
the_table.rozdanie()

# wyświetlenie rąk grających i talii po rozdaniu
for gracz in the_table:
    print(f'{gracz.imię}, {gracz.pozycja} (rozdane):     {gracz.hand()}')
print('')
for gracz in the_table:
    print(f'{gracz.imię}, {gracz.pozycja} (posortowane): {gracz.hand_sorted()}')
print('')
print(f'talia po rozdaniu: {the_table.deck}, pozostało: {the_table.deck.liczba_kart()}')   

Piotr, N (rozdane):     ['W♦️', 'K♦️', '6♦️', '10♣️', '3♦️', 'K♠️', 'W♣️', 'K♥️', 'A♥️', '2♥️', '7♣️', '7♥️', '8♠️']
Agata, E (rozdane):     ['9♣️', '2♦️', '7♦️', '10♠️', '9♥️', '9♦️', '4♦️', '6♥️', 'Q♠️', '5♦️', 'A♣️', '5♥️', '2♣️']
Kasia, S (rozdane):     ['6♣️', 'W♥️', '9♠️', '10♦️', '3♥️', '4♥️', '4♣️', '8♥️', 'A♠️', '6♠️', 'Q♦️', 'W♠️', 'Q♥️']
Karol, W (rozdane):     ['5♣️', 'A♦️', '3♠️', '4♠️', 'Q♣️', '8♣️', '7♠️', '8♦️', '5♠️', '2♠️', 'K♣️', '3♣️', '10♥️']

Piotr, N (posortowane): ['K♠️', '8♠️', 'A♥️', 'K♥️', '7♥️', '2♥️', 'W♣️', '10♣️', '7♣️', 'K♦️', 'W♦️', '6♦️', '3♦️']
Agata, E (posortowane): ['Q♠️', '10♠️', '9♥️', '6♥️', '5♥️', 'A♣️', '9♣️', '2♣️', '9♦️', '7♦️', '5♦️', '4♦️', '2♦️']
Kasia, S (posortowane): ['A♠️', 'W♠️', '9♠️', '6♠️', 'Q♥️', 'W♥️', '8♥️', '4♥️', '3♥️', '6♣️', '4♣️', 'Q♦️', '10♦️']
Karol, W (posortowane): ['7♠️', '5♠️', '4♠️', '3♠️', '2♠️', '10♥️', 'K♣️', 'Q♣️', '8♣️', '5♣️', '3♣️', 'A♦️', '8♦️']

talia po rozdaniu: [], pozostało: 0
